#1. Importing Libraries

In [15]:
import numpy as np
import pandas as pd

In [16]:
c_add=pd.read_excel('Dataset.xlsx',sheet_name='CustomerAddress')

#2. Data Quality Analysis

2.1 Checking records

In [17]:
c_add.head()

,customer_id,address,postcode,state,country,property_valuation
0,1,060 Morning Avenue,2016,New South Wales,Australia,10
1,2,6 Meadow Vale Court,2153,New South Wales,Australia,10
2,4,0 Holy Cross Court,4211,QLD,Australia,9
3,5,17979 Del Mar Point,2448,New South Wales,Australia,4
4,6,9 Oakridge Court,3216,VIC,Australia,9


2.2. Check information of columns and their datatypes

In [18]:
c_add.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3999 entries, 0 to 3998
Data columns (total 6 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   customer_id         3999 non-null   int64 
 1   address             3999 non-null   object
 2   postcode            3999 non-null   int64 
 3   state               3999 non-null   object
 4   country             3999 non-null   object
 5   property_valuation  3999 non-null   int64 
dtypes: int64(3), object(3)
memory usage: 187.6+ KB


Columns arde :customer_id,	address	postcode,	state	country,	property_valuation

#Missing Values

In [19]:
#returns the number of missing values in the dataset
c_add.isnull().sum()

customer_id           0
address               0
postcode              0
state                 0
country               0
property_valuation    0
dtype: int64

#4. Data Inconsistency

4.1 Address

In [20]:
c_add['address'].value_counts()

address
3 Mariners Cove Terrace      2
3 Talisman Place             2
64 Macpherson Junction       2
359 Briar Crest Road         1
4543 Service Terrace         1
                            ..
5063 Shopko Pass             1
09 Hagan Pass                1
87897 Lighthouse Bay Pass    1
294 Lawn Junction            1
320 Acker Drive              1
Name: count, Length: 3996, dtype: int64

4.2 Postcode

In [21]:
c_add['postcode'].value_counts()

postcode
2170    31
2155    30
2145    30
2153    29
3977    26
        ..
3808     1
3114     1
4721     1
4799     1
3089     1
Name: count, Length: 873, dtype: int64

4.3 State

In [22]:
c_add['state'].value_counts()

state
NSW                2054
VIC                 939
QLD                 838
New South Wales      86
Victoria             82
Name: count, dtype: int64

In [23]:
def replace_state(state):

    # Making Short Form of State Names as standards
    if state=='New South Wales':
        return 'NSW'
    elif state=='Victoria':
        return 'VIC'
    else :
        return state

# Applying the above fuction to state column
c_add['state'] = c_add['state'].apply(replace_state)

In [24]:
c_add['state'].value_counts()

state
NSW    2140
VIC    1021
QLD     838
Name: count, dtype: int64

4.5 Country

In [25]:
c_add['country'].value_counts()

country
Australia    3999
Name: count, dtype: int64

In [26]:
# Check for duplicate rows based on specific columns
df=pd.DataFrame(c_add)
duplicate_rows_subset = df.duplicated(subset=['customer_id'])

# Check if there are any duplicates
if duplicate_rows_subset.any():
    print("There are duplicates based on customer_id.")
else:
    print("There are no duplicates based on customer_id.")

There are no duplicates based on customer_id.


# 5. Duplicate

#6. Export

In [27]:
c_add.to_csv('CustomerAddress_Cleaned.csv', index=False)

# 7. Verifying record

We have two data related to customer- one is customer_demographic and another is customer address . we need to verify if there are some customer address which miss maches withh customer in demographic file . If such customer whose details are not found in demographic those address will be meeaningless thus it should be dropped from customer address file .

In [28]:
#importing customer demographic file
c= pd.read_csv('CustomerDemographic_cleaned.csv')

In [29]:
c.head()

,customer_id,first_name,last_name,gender,past_3_years_bike_related_purchases,DOB,job_title,job_industry_category,wealth_segment,deceased_indicator,owns_car,tenure,Age
0,1,Laraine,Medendorp,Female,93,1953-10-12,Executive Secretary,Health,Mass Customer,N,Yes,11.0,70
1,2,Eli,Bockman,Male,81,1980-12-16,Administrative Officer,Financial Services,Mass Customer,N,Yes,16.0,43
2,3,Arlin,Dearle,Male,61,1954-01-20,Recruiting Manager,Property,Mass Customer,N,Yes,15.0,70
3,4,Talbot,NaN,Male,33,1961-10-03,missing,IT,Mass Customer,N,No,7.0,62
4,5,Sheila-kathryn,Calton,Female,56,1977-05-13,Senior Editor,missing,Affluent Customer,N,Yes,8.0,46


In [30]:
c_add.shape[0]


3999

In [31]:
c.shape[0]

3912

Based on cusutomer_id both the files can we merged on full outer join. Thus the rows or customer_id which are not present are deleted while merging as they are invalid. example -customer_id =3   

In [32]:
c_demoadd_updated = c_add.merge(c ,left_on ='customer_id',right_on='customer_id',how='outer')

In [33]:
c_demoadd_updated.shape[0]

4003

In [34]:
c_demo_updated = c_add.merge(c ,left_on ='customer_id',right_on='customer_id',how='right')
c_demo_updated.shape[0]

3912

In [35]:
c_add_updated = c_add.merge(c ,left_on ='customer_id',right_on='customer_id',how='left')
c_add_updated.shape[0]

3999

In [36]:
c_demoadd_updated_common = c_add.merge(c ,left_on ='customer_id',right_on='customer_id',how='inner')
c_demoadd_updated_common.shape[0]

3908

#6. Export

1. We export the updated customer demographic file with both address col and demo column (neglecting the missing vlaues of either of two table due to full outer join) ie. missing address/ or demo field are present
2. We export the updated customer demographic file with address column eliminating the missing address which is not in cust demo file . this will also contain those cust id whose address is missing
3. we also export the updated customer_address file with only valid address columns. This will contain missing demographic details
4. We export the updated customer demographic file with both address col and demo column. No missing value .

In [37]:
c_demoadd_updated.to_csv('Customer_DemoAddr_Updated.csv', index=False)


In [38]:
c_demo_updated.to_csv('Customer_Demo_Updated.csv', index=False)


In [39]:
c_add_updated.to_csv('Customer_Add_Updated.csv', index=False)

In [40]:
c_demoadd_updated_common.to_csv('Customer_Add_Updated_Common.csv', index=False)

Onlyy 4 fields are there in demographic file where address is absent

giving importace to customer demographic = c_demo_updated.csv contains full details of demo+add ie it can contain missing address . 3912 rows